## Setting up parallel training on N GPU/CPU

In [ ]:
#!g1.1 #noqa
import pickle
import random
import sys

import torch.multiprocessing as mp
from torchvision import transforms

In [ ]:
#!g1.1 #noqa
sys.path.append('/home/jupyter/work/resources/')

In [ ]:
#!g1.1 #noqa
from processingDataSet import ImageNetDataset

In [ ]:
#!g1.1 #noqa
with open('../filtered_data_10class.pkl', 'rb') as file:
    train_data = pickle.load(file)
    val_data = pickle.load(file)

In [ ]:
#!g1.1 #noqa
print('Train data size: ', len(train_data), 'Validation data size: ', len(val_data))

In [ ]:
#!g1.1 #noqa
transform_train = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.RandomRotation(random.randint(0, 180))])

In [ ]:
#!g1.1 #noqa
transform_val = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
#!g1.1 #noqa
data_path = '/home/jupyter/mnt/datasets/ImageNet/ILSVRC/Data/CLS-LOC/train'

In [ ]:
#!g1.1 #noqa
train_set = ImageNetDataset(data_path, train_data, transform_train)
val_set = ImageNetDataset(data_path, val_data, transform_val)

In [ ]:
#!g1.1 #noqa
from distLearningFunc import worker

world_size = 4  # Number of CPU or GPU
batch_size = 64
seed = 10
epochs = 3

mp.spawn(worker, args = (world_size, train_set, val_set, batch_size, seed, epochs, False),
         nprocs = world_size)

In [ ]:
#!g1.1 #noqa
'''
    Warning from torch.multiprossesing.spawn:

    If the main process exits abruptly (e.g. because of an incoming signal),
    Python’s multiprocessing sometimes fails to clean up its children.
    It’s a known caveat, so if you’re seeing any resource leaks after interrupting the interpreter,
    it probably means that this has just happened to you.
    https://pytorch.org/docs/stable/multiprocessing.html
'''
# To fix that problem, find the PID of this proсess(es)
# (depends on how many warkers was started) and kill them.
!ps -fe | grep multiprocessing.spawn

In [ ]:
#!g1.1 #noqa
!kill 5836 5837

In [ ]:
#!g1.1 #noqa
